In [2]:
import pandas as pd
import re
from keybert import KeyBERT
import yfinance as yf
import time
import random
from collections import Counter
import numpy as np

news_raw = pd.read_csv("C:/projectnasdaq/news_raw.csv", encoding='latin1')
nasdaq_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks.csv", encoding='latin1')
rep_stock = pd.read_csv("C:/projectnasdaq/nasdaq_stocks_refine_total.csv", encoding='latin1')

news_raw = news_raw.head(1000)
news_raw = news_raw[['news_id','title','summary']]
nasdaq_stock = nasdaq_stock[['pk','symbol','name']]
rep_stock = rep_stock[['pk','symbol','name','name_a']]

# new_raw 데이터 전처리
news_raw['summary'] = news_raw['summary'].str.lower()
news_raw['summary'] = news_raw['summary'].apply(lambda x: re.sub('[^a-zA-Z\d&]', ' ', str(x)).strip())
self_stop_words = {'bloomberg'}
news_raw['summary'].replace(self_stop_words, '', regex=True, inplace=True)

# news_raw 데이터 토큰화
news_raw['tokenize'] = 0
news_raw['tokenize'] = news_raw['summary'].str.split(" ")
news_raw['tokenize'] = news_raw['tokenize'].apply(lambda x: [i for i in x if i != "" and i != " "])

news_raw['company_name'] = 0
news_raw['keyword'] = 0 # keybert로 뽑아낸 키워드 컬럼
news_raw['keyword2'] = 0

# list에 담아주기
tokenize_list = news_raw['tokenize']
rep_list = rep_stock['name_a'].str.split(",")

# 맵핑한 데이터 저장하기 위한 csv파일 생성
project_result = pd.DataFrame(columns=['news_id', 'company_word','pk'])

In [2]:
news_raw

,news_id,title,summary,tokenize,company_name,keyword,keyword2
0,345585,Nestle Outshines Rivals With Revenue Growth Le...,nestle sa sailed past ailing consumer goo...,"[nestle, sa, sailed, past, ailing, consumer, g...",0,0,0
1,345586,Total Makes Surprise Profit as Trading Gains O...,total se made a surprise profit after ve...,"[total, se, made, a, surprise, profit, after, ...",0,0,0
2,345590,Airbus Follows Boeing in Paring Output to Weat...,airbus se cut back wide body jet producti...,"[airbus, se, cut, back, wide, body, jet, produ...",0,0,0
3,345591,Virus Uptick Imperils South Europe's Nascent T...,nadine scheiner s efforts to travel from ...,"[nadine, scheiner, s, efforts, to, travel, fro...",0,0,0
4,345595,PG&E Fire Insurance Costs Skyrocket After Bank...,pg&e corp is finding it very costly to b...,"[pg&e, corp, is, finding, it, very, costly, to...",0,0,0
...,...,...,...,...,...,...,...
995,443773,Localiza to Form $9 Billion Car-Rental Giant W...,localiza rent a car sa and unidas two of...,"[localiza, rent, a, car, sa, and, unidas, two,...",0,0,0
996,443778,Mendoza Province Is First to Restructure After...,argentina s mendoza province has reached ...,"[argentina, s, mendoza, province, has, reached...",0,0,0
997,443779,Manufacturing Keeps German Economy on a Recove...,german factories kept europe s biggest ec...,"[german, factories, kept, europe, s, biggest, ...",0,0,0
998,443780,ECB Must Limit Emergency Powers to Temporary C...,the european central bank risks legal tro...,"[the, european, central, bank, risks, legal, t...",0,0,0


## 회사 이름 추출 => 추출 완료 ( company_word.csv 에 저장 )

In [ ]:
# 대표단어 csv파일에서 대표단어가 포함되면 맵핑
i = 0

for token_num, token in enumerate(tokenize_list):  # news_raw data 토큰화
    for rep_num, rep in enumerate(rep_list):  # 대표단어 csv파일에서 대표단어에 해당
        i = i + 1
        if len(rep) == 1:
            if rep[0] in token:
                project_result.loc[i] = [news_raw['news_id'][token_num], rep[0], rep_stock['pk'][rep_num]]

        elif len(rep) == 2:  # 대표단어가 2개로 된 단어일 때
            if rep[0] in token:  # 대표단어의 첫번째 단어가 org단어에 있으면
                found = token.index(rep[0])  # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치 번호
                try:
                    search = found + 1  # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = token[search]  # org의 (+1을 한) 다음 단어에 해당
                    if rep[1] == search_found:
                        project_result.loc[i] = [news_raw['news_id'][token_num], rep[0] + " " + rep[1],
                                                 rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

        elif len(rep) == 3:
            if rep[0] in token:
                try:
                    search_found = token[token.index(rep[0]) + 1]
                    if rep[1] == search_found:
                        two_found = token[token.index(rep[0]) + 2]
                        if rep[2] == two_found:
                            project_result.loc[i] = [news_raw['news_id'][token_num],
                                                     rep[0] + " " + rep[1] + " " + rep[2], rep_stock['pk'][rep_num]]
                except IndexError:
                    continue

project_result = pd.merge(project_result, nasdaq_stock, how='left', left_on='pk', right_on='pk')

company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')

## 뉴스 원문에서 키워드 추출 ( Keybert 사용 )=> 추출 완료 ( keyword.csv 에 저장 )

In [ ]:
# Keybert로 키워드 추출 ( 키워드 1단어들, 2단어들 )
summaries = news_raw['summary']
# 뉴스 원문 키워드 10개 추출 ( 대표키워드랑 비교용도 )
news_raw['ten_keyword'] = 0

for sum_num, summary in enumerate(summaries):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(summary,top_n=5)
    news_raw['keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1))
    news_raw['keyword2'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 2))
    news_raw['ten_keyword'][sum_num] = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 1),top_n= 10)
#news_raw.to_csv('C:/projectnasdaq/project2/keyword.csv')

## 종목 정보 수집 ( yfinance 패키지 사용 ) => 수집완료

In [ ]:
# 종목코드 리스트
stocks = nasdaq_stock['symbol']
# 종목코드에 대한 정보 수집용 데이터프레임 생성
stock_info = pd.DataFrame(columns=['symbol', 'info'])
i = 0

for stock_num, stock in enumerate(stocks):
    tickers = yf.Ticker(stock)
    ticker = tickers.info
    i = i + 1
    #print(stock_num,stock, "===>>",ticker)
    stock_info.loc[i] = [stock, ticker]
    time.sleep(random.uniform(3, 4))

#stock_info.to_csv('C:/projectnasdaq/project2/stock_info.csv')

## 추출된 회사 이름이랑 ( 전처리 한 ) 키워드 연결 ( company_and_keyword.csv 파일에 저장 )

In [ ]:
company_mapping = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')
keyword_mapping = pd.read_csv('C:/projectnasdaq/project2/keyword.csv')

keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())  # 정규식전처리
keyword_mapping['keyword'] = keyword_mapping['keyword'].apply(lambda x: re.sub(r"\s+", " ", str(x)).strip())  # 공백 여러개 하나로
keyword_mapping['keyword'] = keyword_mapping['keyword'].str.split(" ")
#keyword_mapping['keyword'].replace(' ',',', regex=True, inplace=True)

keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].str.split("  ")
# 공백으로 토큰화되거나 비어있는 내용으로 토큰화 되어있으면 지우기
keyword_mapping['keyword2'] = keyword_mapping['keyword2'].apply(lambda x: [i for i in x if i != "" and i != " "])

company_mapping[['keyword', 'keyword2']] = 0

keyword_list1 = keyword_mapping['keyword']
keyword_list2 = keyword_mapping['keyword2']

keyword_id = keyword_mapping['news_id']
company_id = company_mapping['news_id']

for i, j in enumerate(company_id):
    for k, l in enumerate(keyword_id):
        if j == l:
            company_mapping['keyword'][i] = keyword_mapping['keyword'][k]
            company_mapping['keyword2'][i] = keyword_mapping['keyword2'][k]

#company_mapping.to_csv('C:/projectnasdaq/project2/company_and_keyword.csv')

## yahoo_dataset 파일 수정 ( yahoo_dataset_mapping.csv 에 저장 )

In [95]:
# yahoo_dataset 파일 수정 ( -> symbol name 수정 )
yahoo_dataset = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset.csv')
yahoo_dataset.insert(1,'new_symbol','0')
yahoo_dataset.insert(2,'name','0')
yahoo_dataset.insert(3,'company_word','0')

yahoo_dataset_symbol = yahoo_dataset['symbol']
rep_stock_symbol = rep_stock['symbol']

for i, j in enumerate(yahoo_dataset_symbol):
    for k, l in enumerate(rep_stock_symbol):
        if j == l:
            yahoo_dataset['new_symbol'][i] = rep_stock['symbol'][k]
            yahoo_dataset['name'][i] = rep_stock['name'][k]
            yahoo_dataset['company_word'][i] = rep_stock['name_a'][k]
            break

# yahoo_dataset 컬럼 정리
yahoo_dataset.drop(columns=["quoteType", "currency", 'regularMarketPrice', 'regularMarketChange', 'regularMarketChangePercent',
             'regularMarketVolume', 'averageDailyVolume3Month', 'marketCap', 'trailingPE', 'fiftyTwoWeekLow',
             'fiftyTwoWeekHigh', 'regularMarketOpen', 'priceHint', 'underlyingSymbol'], inplace=True)
yahoo_dataset.to_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')

## 종목명이 추출된 뉴스 키워드들끼리 모으기 ( 딕셔너리 형태로 )

In [42]:
# company단어들 하나씩만 list에 담기 -> 단어 이름이 같은 종목명들의 키워드 모으기 위해서
company_and_keyword = pd.read_csv('C:/projectnasdaq/project2/company_and_keyword.csv')
company_and_keyword.drop(columns=["Unnamed: 0", 'Unnamed: 0.1'], inplace=True)
company_word = company_and_keyword['company_word']

company_word_same_list = []
for company in company_word:
    if company not in company_word_same_list:
        company_word_same_list.append(company)

company_word_sames = pd.DataFrame(company_word_same_list, columns=['company_word_same'])
company_word_same = company_word_sames['company_word_same']

# 키워드랑 company_name이랑 데이터프레임에서 맵핑 시키기위한 새로운 dataframe 생성
company__name = pd.DataFrame(columns=['company', 'keyword', 'keyword2','key_list'])

# 딕셔너리를 { news_id : keyword } 형태로 만들기 위해서 index를 news_id로 설정
company_and_keyword = company_and_keyword.set_index(keys='news_id', drop=False, inplace=False)

# company__name에 대한 키워드들 딕셔너리 형태로 모으기
for i, j in enumerate(company_word_same):
    company__name.loc[i] = [j, company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].to_dict(), company_and_keyword[company_and_keyword['company_word'] == j]['keyword2'].to_dict(),company_and_keyword[company_and_keyword['company_word'] == j]['keyword'].tolist()] # 리스트 형태로 키워드만 뽑아오기 위해서

## news_raw_dataset 만들기

In [43]:
# company__name.csv에 column 삽입
company__name.insert(0, 'symbol','0')
company__name.insert(1, 'name','0')
company__name.insert(2, 'shortName','0')
company__name.insert(3, 'longName','0')
company__name.insert(5, 'company_info','0')

yahoo_dataset_mapping = pd.read_csv('C:/projectnasdaq/project2/yahoo_dataset_mapping.csv')
company_word = pd.read_csv('C:/projectnasdaq/project2/company_word.csv')

# company__name.csv에 symbol, name 데이터 삽입
company_word_ = company_word['company_word']
company__ = company__name['company']

for i,j in enumerate(company_word_):
    for k, l in enumerate(company__):
        if j==l:
            company__name['symbol'][k] = company_word['symbol'][i]
            company__name['name'][k] = company_word['name'][i]
            break

# company__name.csv에 yahoo finance 정보 수집 내용 삽입
yahoo_dataset_mapping_symbol = yahoo_dataset_mapping['new_symbol']
company__name_symbol = company__name['symbol']

for i,j in enumerate(yahoo_dataset_mapping_symbol):
    for k,l in enumerate(company__name_symbol):
        if j==l:
            company__name['shortName'][k] = yahoo_dataset_mapping['shortName'][i]
            company__name['longName'][k] = yahoo_dataset_mapping['longName'][i]
            company__name['company_info'][k] = yahoo_dataset_mapping['company_info'][i]
            break

# 완성된 news_raw dataset
#company__name.to_csv('C:/projectnasdaq/project2/news_raw_dataset.csv')

## 대표 키워드 추출

In [258]:
# 키워드 빈도수 순으로 정리해서 5개까지만 뽑아오기
company__name['frequency_word'] = 0
company__name['frequency'] = 0

key_list_ = company__name['key_list']
for i, j in enumerate(key_list_):
        company__name['frequency_word'][i] = ''.join(j)

company__name['frequency_word'].replace(['\]','\[','\''],'',regex=True, inplace=True)
company__name['frequency_word'].replace(',',' ',regex=True, inplace=True)
company__name['frequency_word'] = company__name['frequency_word'].str.split(" ")
company__name['frequency_word'] = company__name['frequency_word'].apply(lambda x: [i for i in x if i != "" and i != " "])

frequency_words = company__name['frequency_word']
for i, j in enumerate(frequency_words):
    # 대표 키워드 빈도순으로 5개 추출
    company__name['frequency'][i] = Counter(j).most_common(5)

# 다섯개 뽑아온 키워드 전처리
company__name['frequency'] = company__name['frequency'].apply(lambda x: re.sub('[^a-zA-Z&]', ' ', str(x)).strip())
company__name['frequency'] = company__name['frequency'].str.split(" ")
company__name['frequency'] = company__name['frequency'].apply(lambda x: [i for i in x if i != "" and i != " "])

C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\q1035\anaconda3\envs\intern_lv2\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [262]:
# 뉴스 원문에 대표 키워드가 포함될 경우 news_id와 symbol 맵핑
frequency = company__name['frequency'] # 키워드 빈도수로 정렬 후 5개만 담아온 리스트
news_stock_mapping = pd.DataFrame(columns=['news_id', 'symbol','name','summary','frequency'])

a = 0
for i, j in enumerate(tokenize_list):
    for k,l in enumerate(frequency):
        a = a + 1
        #print(news_raw['news_id'][i])
        together = set(j)&set(l)
        # 뉴스 원문에 대표 키워드가 4개 이상 교집합으로 있을 경우
        if len(together)==5:
            news_stock_mapping.loc[a] = [news_raw['news_id'][i], company__name['symbol'][k], company__name['name'][k],                                                 news_raw['summary'][i], company__name['frequency'][k]]

In [264]:
print('총 맵핑 수 :',len(news_stock_mapping))

총 맵핑 수 : 544
